In [1]:
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
#读取文件  
import os 
import csv
for root, dirs, files in os.walk('dataset'):  
        print(root) #当前目录路径  
        print(files) #当前路径下所有非目录子文件
#进入dataset文件
os.chdir('dataset')

dataset
['ant-1.3.csv', 'ant-1.4.csv', 'ant-1.5.csv', 'ant-1.6.csv', 'ant-1.7.csv', 'camel-1.0.csv', 'camel-1.2.csv', 'camel-1.4.csv', 'camel-1.6.csv', 'ivy-1.1.csv', 'ivy-1.4.csv', 'ivy-2.0.csv', 'jedit-3.2.csv', 'jedit-4.0.csv', 'jedit-4.1.csv', 'jedit-4.2.csv', 'jedit-4.3.csv', 'log4j-1.0.csv', 'log4j-1.1.csv', 'log4j-1.2.csv', 'lucene-2.0.csv', 'lucene-2.2.csv', 'lucene-2.4.csv', 'poi-1.5.csv', 'poi-2.0.csv', 'poi-2.5.csv', 'poi-3.0.csv', 'synapse-1.0.csv', 'synapse-1.1.csv', 'synapse-1.2.csv', 'velocity-1.4.csv', 'velocity-1.5.csv', 'velocity-1.6.csv', 'xalan-2.4.csv', 'xalan-2.5.csv', 'xalan-2.6.csv', 'xalan-2.7.csv', 'xerces-1.2.csv', 'xerces-1.3.csv', 'xerces-1.4.csv']


In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import sklearn.model_selection as sk_model_selection
from hyperopt import fmin,tpe,hp,partial,anneal,rand
from sklearn import metrics
import time
from hyperopt.early_stop import no_progress_loss

In [3]:
#写入文件函数
def writercsv(file_path,param,G):
    if os.path.isfile(file_path):
        with open(file_path,'a',newline='')as f:
            csv_write=csv.writer(f,dialect='excel')
            csv_write.writerow(G)
    else:
        with open(file_path,'w',newline='')as f:
            csv_write=csv.writer(f,dialect='excel')
            csv_write.writerow(param)
            csv_write.writerow(G)
#文件路径
file_path='..\Rand_KNN\Rand_KNN_10(5).csv'
param=['Precision','Recall','F-measure','AUC','n_neighbors','weights',
       'MSE_default','MSE_TPE','Bias_Default','Bias_TPE','var_Default','var_TPE','Time']

In [4]:
#顺序读取文件进行实验
num=0
while num<39:
    #读取文件
    df1=pd.read_csv(files[num],header=0)
    df2=pd.read_csv(files[num+1],header=0)
    print(files[num],"->",files[num+1])
    #调参实验
    x_train=df1.iloc[:,0:19].values
    y_train=df1.iloc[:,20].values
    x_test=df2.iloc[:,0:19].values
    y_true=df2.iloc[:,20].values  
     #特征选择
    from skfeature.function.statistical_based import CFS
    feature_index = CFS.cfs(x_train,y_train) # 在训练集上训练
    num_feature = 10 # 从原数据集中选择10个特征
    t_train = x_train[:,feature_index[0:num_feature]] # 转换训练集
    t_test = x_test[:,feature_index[0:num_feature]] # 转换测试集
    def KNN(args):
        hKN=KNeighborsClassifier(n_neighbors=int(args['n_neighbors']),weights=args['weights']) 
        auc=sk_model_selection.cross_val_score(hKN,t_train,y_train,scoring='roc_auc',cv=10)
        return -auc.mean()
    #参数空间
    space={'n_neighbors':hp.quniform('n_neighbors',1,50,2),
      'weights':hp.choice('weights',['uniform','distance'])}
    #代理函数
    #algo=partial(tpe.suggest)
    algo=partial(rand.suggest)
    #algo=partial(anneal.suggest)
    time_start = time.process_time()
    best = fmin(KNN,space=space,algo = algo,max_evals=200)
    time_end = time.process_time()  # 记录结束时间
    time_sum = time_end - time_start  # 计算的时间差为程序的执行时间，单位为秒/s
    #设置离散变量
    if best['weights']==0:
        best['weights']='uniform'
    else: 
        best['weights']='distance'
    #参数带回实验进行预测
    model=KNeighborsClassifier(n_neighbors=int(best['n_neighbors']),weights=best['weights'])
    model.fit(t_train,y_train)
    y_pred=model.predict(t_test)
    #评分
    G=[]
    probas=model.predict_proba(t_test)
    fpr,tpr,thresholds=roc_curve(y_true,probas[:,1])
    auc=metrics.auc(fpr,tpr)
    f_measure=metrics.f1_score(y_true,y_pred,average='weighted')
    recall=metrics.recall_score(y_true,y_pred,average='weighted')
    precision=metrics.precision_score(y_true,y_pred,average='weighted')
    G.append(precision)
    G.append(recall)
    G.append(f_measure)
    G.append(auc)
    G.append(best['n_neighbors'])
    G.append(best['weights'])
    #计算模型稳定性
    model_d=KNeighborsClassifier()
    model_d.fit(x_train,y_train)
    from mlxtend.evaluate import bias_variance_decomp
    MSE_Default,Bias_Default,var_Default=bias_variance_decomp(model_d,t_train,y_train,t_test,y_true,
                                  loss='mse',num_rounds=100,random_seed=1)
    MSE_TPE,Bias_TPE,var_TPE=bias_variance_decomp(model,t_train,y_train,t_test,y_true,
                                  loss='mse',num_rounds=100,random_seed=1)
    G.append(MSE_Default)
    G.append(MSE_TPE)
    G.append(Bias_Default)
    G.append(Bias_TPE)
    G.append(var_Default)
    G.append(var_TPE)
    #记录时间
    G.append(time_sum)
    #写入文件
    writercsv(file_path,param,G)
    print('写入文件成功')
    #版本变化
    num=num+1
    #项目变化
    if((num==4) or (num==8)or(num==11)or(num==16)or(num==19)or(num==22)or(num==26)or(num==29)or(num==32)or(num==36)):
        num=num+1

ant-1.3.csv -> ant-1.4.csv
100%|█████████████████████████████████████████████| 200/200 [00:03<00:00, 65.88trial/s, best loss: -0.8470454545454545]
写入文件成功
ant-1.4.csv -> ant-1.5.csv
100%|█████████████████████████████████████████████| 200/200 [00:03<00:00, 60.64trial/s, best loss: -0.5908791208791209]
写入文件成功
ant-1.5.csv -> ant-1.6.csv
100%|█████████████████████████████████████████████| 200/200 [00:03<00:00, 56.43trial/s, best loss: -0.7491690408357075]
写入文件成功
ant-1.6.csv -> ant-1.7.csv
100%|█████████████████████████████████████████████| 200/200 [00:03<00:00, 53.69trial/s, best loss: -0.8312982905982904]
写入文件成功
camel-1.0.csv -> camel-1.2.csv
100%|█████████████████████████████████████████████| 200/200 [00:03<00:00, 55.99trial/s, best loss: -0.6904592803030303]
写入文件成功
camel-1.2.csv -> camel-1.4.csv
100%|██████████████████████████████████████████████| 200/200 [00:04<00:00, 44.57trial/s, best loss: -0.584438616938617]
写入文件成功
camel-1.4.csv -> camel-1.6.csv
100%|████████████████████████████████